In [7]:
#Change directory to data location
import os
path = 'E:\Daily Reanalisis ERA 5 v2'
os.chdir(path)

In [8]:
import warnings
warnings.simplefilter('ignore') #ignores simple warning

In [9]:
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4 as nc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [10]:
erossby=xr.open_dataset('E:\CobaWK99\Data Hasil Filter\wk_er_olr.nc')
mrossby=xr.open_dataset('E:\CobaWK99\Data Hasil Filter\wk_mrg_olr.nc')

In [35]:
idxer1=erossby.sel(lat= slice(3,14), lon=slice(86,101), time=slice('1979-01-01T00:00:00.000000000','2020-12-31T00:00:00.000000000')).mean(dim=('lat','lon'))['olr_er']
idxer2=erossby.sel(lat= slice(-16,-4), lon=slice(113,128), time=slice('1979-01-01T00:00:00.000000000','2020-12-31T00:00:00.000000000')).mean(dim=('lat','lon'))['olr_er']
idxer=idxer1-idxer2

In [36]:
mrg1 = mrossby.sel(lat= slice(2,9), lon=slice(97,110), time=slice('1979-01-01T00:00:00.000000000','2020-12-31T00:00:00.000000000')).mean(dim=('lat','lon'))['olr_mrg']
mrg2 = mrossby.sel(lat= slice(-6,-3), lon=slice(105,117), time=slice('1979-01-01T00:00:00.000000000','2020-12-31T00:00:00.000000000')).mean(dim=('lat','lon'))['olr_mrg']
idxmrg=mrg1-mrg2

In [37]:
time=idxer['time'][1:-1].values

In [38]:
dtidxmrg=(idxmrg[2:].values-idxmrg[:-2].values)/2
dtidxer=(idxer[2:].values-idxer[:-2].values)/2

In [39]:
idxmrgcut=idxmrg[1:-1].values
idxercut=idxer[1:-1].values

In [40]:
#Normalization
idxmrgcut=idxmrgcut/np.std(idxmrgcut)
idxercut=idxercut/np.std(idxercut)
dtidxmrg=dtidxmrg/np.std(dtidxmrg)
dtidxer=dtidxer/np.std(dtidxer)

In [41]:
#Change directory to data location
import os
path = 'E:\Daily Reanalisis ERA 5 v2'
os.chdir(path)

In [42]:
ampmrg=np.sqrt(dtidxmrg**2+idxmrgcut**2)
amper=np.sqrt(dtidxer**2+idxercut**2)

In [43]:
angmrg=np.rad2deg(np.arctan2(dtidxmrg,idxmrgcut))
anger=np.rad2deg(np.arctan2(dtidxer,idxercut))

In [44]:
phmrg=np.empty([len(angmrg)])
for i in range(len(angmrg)):
    if (angmrg[i]<=45 and angmrg[i]>-45):
        phmrg[i]=1
    elif (angmrg[i]<=-45 and angmrg[i]>-135):
        phmrg[i]=2
    elif (angmrg[i]<=-135 or angmrg[i]>135):
        phmrg[i]=3
    elif (angmrg[i]<=135 and angmrg[i]>45):
        phmrg[i]=4

In [45]:
pher=np.empty([len(anger)])
for i in range(len(anger)):
    if (anger[i]<=45 and anger[i]>-45):
        pher[i]=1
    elif (anger[i]<=-45 and anger[i]>-135):
        pher[i]=2
    elif (anger[i]<=-135 or anger[i]>135):
        pher[i]=3
    elif (anger[i]<=135 and anger[i]>45):
        pher[i]=4

In [46]:
tglidxermrg=pd.DataFrame({'time': time,
                         'phase mrg':phmrg,
                         'amp mrg'  :ampmrg,
                         'phase er' :pher,
                         'amp er'   :amper,
                          })
tglidxermrg

,time,phase mrg,amp mrg,phase er,amp er
0,1979-01-02,1.0,0.051840,3.0,0.118740
1,1979-01-03,2.0,0.057774,3.0,0.122629
2,1979-01-04,3.0,0.065331,4.0,0.132772
3,1979-01-05,3.0,0.063530,4.0,0.140729
4,1979-01-06,4.0,0.046597,4.0,0.139911
...,...,...,...,...,...
15334,2020-12-26,2.0,0.529067,2.0,1.003094
15335,2020-12-27,2.0,0.485509,2.0,0.521547
15336,2020-12-28,3.0,0.411978,3.0,0.392161
15337,2020-12-29,4.0,0.361821,4.0,0.619055


In [47]:
writer = pd.ExcelWriter('calculation_output\Indeks ER MRG Double Side ER EWNS.xlsx')
tglidxermrg.to_excel(writer, 'CENS_event')
writer.close()